# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool

import chromadb
from lib.state_machine import Run, Resource
from typing import List, TypedDict
from chromadb.api.models.Collection import Collection
from lib.vector_db import VectorStoreManager, CorpusLoaderService
from lib.rag import RAG

from chromadb.utils import embedding_functions

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created

embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY)
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection(name="udaplay", embedding_function=embedding_fn)
# Need to pass embedding function so dimensions match
# The collection was created with an OpenAI embedding model, which has a dimension size of 1536.
# However, it appears that Chroma uses an embedding model with a dimension size of 384 to embed the query.
# So, if the embedding_function is not passed to get_collection(),  and then
# the query() method is called on the collection, this will result in:
# chromadb.errors.InvalidArgumentError: Collection expecting embedding with dimension of 1536, got 384
# Essentially, the dimensions don't match, and passing embedding_function to get_collections() fixes this.

# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

@tool
def retrieve_game(query: str):
    results = collection.query(
    query_texts=[query],
    n_results=3,
    include=['metadatas']
    )
    return results['metadatas'][0] 

In [6]:
collection.peek() # confirm collection has been successfully retrieved

{'ids': ['001', '002', '003', '004', '005', '006', '007', '008', '009', '010'],
 'embeddings': array([[-0.00270652, -0.01438423, -0.0094357 , ..., -0.02086851,
         -0.00733747, -0.03789448],
        [-0.00149212, -0.02129163,  0.00150162, ..., -0.04491242,
         -0.00209182,  0.00053008],
        [-0.01168427, -0.00465201,  0.00187931, ..., -0.02636779,
         -0.00329597, -0.0358122 ],
        ...,
        [ 0.01998285, -0.05459335,  0.00992325, ..., -0.02266045,
         -0.0161152 ,  0.00111179],
        [ 0.00100258, -0.0505378 ,  0.0078321 , ..., -0.01040299,
         -0.01144015, -0.01337618],
        [-0.01217713, -0.02789508, -0.00563168, ..., -0.0383018 ,
         -0.00480359, -0.00478138]], shape=(10, 1536)),
 'documents': ['[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.',
  "[PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in t

In [7]:
question = "I like to drive. What games would I like?"
answer = retrieve_game(question)

answer

[{'Platform': 'PlayStation 3',
  'Publisher': 'Sony Computer Entertainment',
  'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
  'YearOfRelease': 2010,
  'Name': 'Gran Turismo 5',
  'Genre': 'Racing'},
 {'YearOfRelease': 1997,
  'Name': 'Gran Turismo',
  'Publisher': 'Sony Computer Entertainment',
  'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.',
  'Platform': 'PlayStation 1',
  'Genre': 'Racing'},
 {'Platform': 'PlayStation 2',
  'Genre': 'Action-adventure',
  'Publisher': 'Rockstar Games',
  'YearOfRelease': 2004,
  'Description': "An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.",
  'Name': 'Grand Theft Auto: San Andreas'}]

#### Evaluate Retrieval Tool

In [ ]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result



evaluation_prompt = """Your task is to evaluate if the documents are enough to respond the query. 
Give a detailed explanation, so it's possible to take an action to accept it or not."""

@tool
def evaluate_retrieval(question: str, retrieved_docs: list[str]):
    

#### Game Web Search Tool

In [ ]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

### Agent

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

In [ ]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes